In [ ]:
! pip install scipy sklearn datasets==2.5.1 transformers==4.22.2 

In [1]:
import torch
import numpy as np
import datasets
import pickle
import pathlib
import os
import sys
from importlib import reload
import torch.nn as nn
import torch.optim as optim
from tqdm.auto import tqdm
from importlib import reload
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
from transformers.trainer_pt_utils import LengthGroupedSampler
from transformers import AutoModelForSequenceClassification, TrainingArguments

module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    print(module_path)
    sys.path.append(module_path)
# from globenc_extension.src.frozen_training.trainer import Trainer

/home/modaresi/projects


In [2]:
MODEL_CHECKPOINT = "bert-base-uncased"
# MODEL_CHECKPOINT = "/home/modaresi/projects/globenc_extension/outputs/output_mrpc_bert-base-uncased_0001_SEED0042/checkpoint-115"
# MODEL_CHECKPOINT = "/home/modaresi/projects/globenc_extension/outputs/output_mrpc_bert-base-uncased_0001_SEED0042/checkpoint-230"
# MODEL_CHECKPOINT = "/home/modaresi/projects/globenc_extension/outputs/output_mrpc_bert-base-uncased_0001_SEED0042/checkpoint-575"

# MODEL_CHECKPOINT = "/home/modaresi/projects/globenc_extension/outputs/output_cola_bert-base-uncased_0001_SEED0042/checkpoint-268"
# MODEL_CHECKPOINT = "/home/modaresi/projects/globenc_extension/outputs/output_cola_bert-base-uncased_0001_SEED0042/checkpoint-536"

# MODEL_CHECKPOINT = "/home/modaresi/projects/globenc_extension/outputs/output_sst2_bert-base-uncased_0001_SEED0042/checkpoint-2105"

# MODEL_CHECKPOINT = "/home/modaresi/projects/globenc_extension/outputs/output_mnli_bert-base-uncased_0001_SEED0042/checkpoint-12272"

TASK = "mnli"
BATCH_SIZE = 32
LEARNING_RATE = 3e-05
MAX_LENGTH = 128
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cuda'

# Dataset

In [3]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]
actual_task = "mnli" if TASK == "mnli-mm" else TASK
dataset = datasets.load_dataset("glue", actual_task)
metric = datasets.load_metric('glue', actual_task)
metric_name = "pearson" if TASK == "stsb" else "matthews_correlation" if TASK == "cola" else "accuracy"
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}
sentence1_key, sentence2_key = task_to_keys[TASK]
validation_key = "validation_mismatched" if TASK == "mnli-mm" else "validation_matched" if TASK == "mnli" else "validation"
dataset["validation"] = dataset[validation_key]
num_labels = 3 if TASK.startswith("mnli") else 1 if TASK=="stsb" else 2

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if TASK != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT, use_fast=True)
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

encoded_dataset = dataset.map(preprocess_function, batched=True)

print(TASK, sentence1_key, sentence2_key, validation_key, metric_name, num_labels)
encoded_dataset

/home/modaresi/.conda/envs/mohsen/lib/python3.7/site-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)
Found cached dataset glue (/opt/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/5 [00:00<?, ?it/s]

/home/modaresi/.conda/envs/mohsen/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  after removing the cwd from sys.path.
Loading cached processed dataset at /opt/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-e8584dad1d4db399.arrow
Loading cached processed dataset at /opt/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-887e0f329be198d1.arrow


  0%|          | 0/10 [00:00<?, ?ba/s]

Loading cached processed dataset at /opt/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-7187f5948bbcf9d2.arrow
Loading cached processed dataset at /opt/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-38f7fe486997660b.arrow
Loading cached processed dataset at /opt/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-ef0117ce1308704b.arrow


mnli premise hypothesis validation_matched accuracy 3


DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9847
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'

# Model

In [4]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_CHECKPOINT, num_labels=num_labels)
# for name, param in model.base_model.state_dict().items():
#     if "pooler" not in name:
#         print(name)
#         param.requires_grad = False
# model.train()
# for name, param in model.state_dict().items():
#     if "pooler" in name or "classifier" in name:
#         param.requires_grad = True
#     else:
#         param.requires_grad = False
#     print(name, param.requires_grad)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

# Train

In [5]:
import globenc_extension.src.frozen_training.frozen_trainer
reload(globenc_extension.src.frozen_training.frozen_trainer)

args = TrainingArguments(
    f"cls-finetuned-{TASK}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=10,
#     weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    group_by_length=True,
    overwrite_output_dir=True,
    logging_steps=250,
)
trainer = globenc_extension.src.frozen_training.frozen_trainer.Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
print(model.bert.encoder.layer[11].output.dense.bias[:10])
print(MODEL_CHECKPOINT)
trainer.train()
print(model.bert.encoder.layer[11].output.dense.bias[:10])

tensor([-0.0657,  0.0421,  0.0120,  0.0023,  0.0555,  0.0459,  0.0220, -0.0552,
        -0.0217, -0.1011], device='cuda:0', grad_fn=<SliceBackward0>)
bert-base-uncased


Optimizing Params:
['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'classifier.weight', 'classifier.bias']
/home/modaresi/.conda/envs/mohsen/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.964800,0.944697,0.554967
2,0.952700,0.932050,0.564952
3,0.960300,0.930607,0.563220
4,0.954100,0.928367,0.569435
5,0.956100,0.929218,0.561793


Configuration saved in cls-finetuned-mnli/checkpoint-12272/config.json
Model weights saved in cls-finetuned-mnli/checkpoint-12272/pytorch_model.bin
tokenizer config file saved in cls-finetuned-mnli/checkpoint-12272/tokenizer_config.json
Special tokens file saved in cls-finetuned-mnli/checkpoint-12272/special_tokens_map.json
Configuration saved in cls-finetuned-mnli/checkpoint-49088/config.json
Model weights saved in cls-finetuned-mnli/checkpoint-49088/pytorch_model.bin
tokenizer config file saved in cls-finetuned-mnli/checkpoint-49088/tokenizer_config.json
Special tokens file saved in cls-finetuned-mnli/checkpoint-49088/special_tokens_map.json
Configuration saved in cls-finetuned-mnli/checkpoint-61360/config.json
Model weights saved in cls-finetuned-mnli/checkpoint-61360/pytorch_model.bin
tokenizer config file saved in cls-finetuned-mnli/checkpoint-61360/tokenizer_config.json
Special tokens file saved in cls-finetuned-mnli/checkpoint-61360/special_tokens_map.json
ERROR:root:Internal Py

Traceback (most recent call last):
  File "/home/modaresi/.conda/envs/mohsen/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2277949/4161133438.py", line 3, in <module>
    trainer.train()
  File "/home/modaresi/projects/globenc_extension/src/frozen_training/frozen_trainer.py", line 1545, in train
    ignore_keys_for_eval=ignore_keys_for_eval,
  File "/home/modaresi/projects/globenc_extension/src/frozen_training/frozen_trainer.py", line 1783, in _inner_training_loop
    tr_loss_step = self.training_step(model, inputs)
  File "/home/modaresi/projects/globenc_extension/src/frozen_training/frozen_trainer.py", line 2537, in training_step
    loss.backward()
  File "/home/modaresi/.conda/envs/mohsen/lib/python3.7/site-packages/torch/_tensor.py", line 396, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph, inputs=inputs)
  File "/home/modaresi/.c

In [ ]:
model